Unifying Datasets Remote
========

In [1]:
import pandas as pd
import numpy as np
from glob import glob

In [2]:
files = glob('../inmet_datasets/*.csv')

In [3]:
# replaces every ';' with ',' and ' ' with '_', and creates a new file

def rewrite_file(file_name):
    with open('../pre-processed_datasets/' + file_name.split('datasets/')[1], 'w') as new_file:
        with open(file_name, 'r+') as file:            
            for line in file:
                line = line.replace(';', ',')
                line = line.replace(' ', '_')
                line = line[: -2]
                
                new_file.write(line + '\n')
        
            file.close()
        
        new_file.close()
        

In [4]:
def merge_rows (dataset):
    """"
    Function responsible for normalizing the dataset
    
    It receives a pandas dataframe, and, for each row where the column 'Hora' is 1200,
    it takes the 'Precipitacao' and 'TempMinima' columns and puts its values on the
    row above, for it does not have such values (NaN value).
    """
    for index, row in dataset.iterrows():
        if row.loc['Hora'] == 1200:
            precipitacao = row.loc['Precipitacao']
            temp_minima = row.loc['TempMinima']

            dataset.at[index - 1, 'Precipitacao'] = precipitacao
            dataset.at[index - 1, 'TempMinima'] = temp_minima

In [11]:
def precipitation_rate(dataset, min_len):
    r0 = len(dataset[dataset['Precipitacao'] == 0])
    r1 = len(dataset[dataset['Precipitacao'] == 1])
    
    r0 = r0/min_len
    r1 = r1/min_len
    
    print(r0)
    print(r1)
    
    df0 = dataset[dataset['Precipitacao'] == 0].sample(frac=r0)
    df1 = dataset[dataset['Precipitacao'] == 1].sample(frac=r1)
    
    return pd.merge(df0, df1, on=['Estacao', 'Data', 'Hora', 'Precipitacao', 'TempMaxima', 'TempMinima', 'Insolacao', 'Evaporacao_Piche', 'Temp_Comp_Media', 'Umidade_Relativa_Media', 'Velocidade_do_Vento_Media'], how='outer')

In [6]:
dataset_min_len = []

for file in files:
    rewrite_file(file)
    
    csv_file = pd.read_csv('../pre-processed_datasets/' + file.split('datasets/')[1])
    
    dates = csv_file['Data'].unique()
    
    # deleting unique dates
    for date in dates:
        if (len(csv_file[csv_file['Data'] == str(date)]) < 2):
            csv_file = csv_file[csv_file['Data'] != str(date)]
    
    # enconding 'Precipitacao' into 0 or 1
    csv_file['Precipitacao'] = csv_file['Precipitacao'].apply(lambda x: 1 if x > 0 else 0)
    
    merge_rows(csv_file)
    
    csv_file.dropna(how='any', inplace=True)
    
    dataset_min_len.append(len(csv_file))
    
    # saving file
    csv_file.to_csv('../pre-processed_datasets/' + file.split('datasets/')[1], index=False)

In [8]:
unified = pd.DataFrame(columns=['Estacao', 'Data', 'Hora', 'Precipitacao', 'TempMaxima', 'TempMinima', 'Insolacao', 'Evaporacao_Piche', 'Temp_Comp_Media', 'Umidade_Relativa_Media', 'Velocidade_do_Vento_Media'])

In [12]:
dataset_min_len = np.amin(dataset_min_len)

for file in files:
    csv_file = pd.read_csv('../pre-processed_datasets/' + file.split('datasets/')[1])
    dataframe = precipitation_rate(csv_file, dataset_min_len)
    unified = pd.merge(unified, dataframe_file, on=['Estacao', 'Data', 'Hora', 'Precipitacao', 'TempMaxima', 'TempMinima', 'Insolacao', 'Evaporacao_Piche', 'Temp_Comp_Media', 'Umidade_Relativa_Media', 'Velocidade_do_Vento_Media'], how='outer')

inf
inf


/home/gustavo/anaconda3/envs/prototipo/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in long_scalars
  """
/home/gustavo/anaconda3/envs/prototipo/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in long_scalars
  


OverflowError: cannot convert float infinity to integer

In [ ]:
# backup function

# def unify_datasets():
#     for file in files:
#         file_name = '../pre-processed_dataset/' + file.split('datasets/')[1]
#         dataframe_file = pd.read_csv(file_name)
#         unified = pd.merge(unified, dataframe_file, on=['Estacao', 'Data', 'Hora', 'Precipitacao', 'TempMaxima', 'TempMinima', 'Insolacao', 'Evaporacao_Piche', 'Temp_Comp_Media', 'Umidade_Relativa_Media', 'Velocidade_do_Vento_Media'], how='outer')
#     unified.to_csv('../unified.csv', index=False)